In [1]:
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import html
import time

In [ ]:
!pip install ultralytics

In [3]:
def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  image_bytes = b64decode(js_reply.split(',')[1])
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

In [4]:
face_cascade = cv2.CascadeClassifier(cv2.samples.findFile(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'))

In [11]:
def take_photo(filename='photo.jpg', quality=0.8):
    js = Javascript('''
        async function takePhoto(quality) {
          const div = document.createElement('div');
          const capture = document.createElement('button');
          capture.textContent = 'Capture';
          div.appendChild(capture);

          const video = document.createElement('video');
          video.style.display = 'block';
          const stream = await navigator.mediaDevices.getUserMedia({video: true});

          document.body.appendChild(div);
          div.appendChild(video);
          video.srcObject = stream;
          await video.play();

          // Resize the output to fit the video element.
          google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

          // Wait for Capture to be clicked.
          await new Promise((resolve) => capture.onclick = resolve);

          // Create canvas and set dimensions to 640x640
          const canvas = document.createElement('canvas');
          canvas.width = 640;
          canvas.height = 640;

          // Resize the video to fit the canvas size (cropping may occur)
          const context = canvas.getContext('2d');
          context.drawImage(video, 0, 0, 640, 640);  // Draw the video on the 640x640 canvas

          stream.getVideoTracks()[0].stop();
          div.remove();
          return canvas.toDataURL('image/jpeg', quality);
        }
    ''')
    display(js)

    data = eval_js('takePhoto({})'.format(quality))
    # get OpenCV format image
    img = js_to_image(data)
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    print(gray.shape)
    faces = face_cascade.detectMultiScale(gray)
    # Save the grayscale image
    cv2.imwrite(filename, gray)

    return filename

In [14]:
try:
  filename = take_photo('photo.jpg')
  print('Saved to {}'.format(filename))
except Exception as err:
  print(str(err))

<IPython.core.display.Javascript object>

(640, 640)
Saved to photo.jpg


In [15]:
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt

# Load the trained model
model = YOLO('/content/new_trained_model_face.pt')

# Load the image
image_path = filename  # Provide the path to your image
image = cv2.imread(image_path)
image = cv2.resize(image, (256,256)) # Use cv2.resize to resize the image

# Perform prediction
results = model(image)
results = results[0]
results.show()
results.save('imh.jpg')


0: 640x640 1 Himanshu, 20.9ms
Speed: 2.7ms preprocess, 20.9ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 640)


'imh.jpg'